<a href="https://colab.research.google.com/github/akkkiii08/Deep-Learning/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recurrent Neural Network**

We will explore the concept of the Recurrent Neural Network.

Recurrent Neural Networks (RNNs) are a type of neural network designed for processing sequences of data. Unlike traditional neural networks, which assume that inputs are independent of each other, RNNs consider the order and dependencies between elements in a sequence. This makes them particularly useful for tasks like language modeling, speech recognition, and time series prediction.

In an RNN, information cycles through a loop, allowing it to maintain a memory of previous inputs and make decisions based on the context of the entire sequence. However, RNNs can struggle with long-term dependencies, which has led to the development of more advanced architectures like LSTM (Long Short-Term Memory) and GRU (Gated Recurrent Unit) networks that better handle these challenges.

We will take an program which will help to understand how the RNNs works.This program will cover the basic structure and steps involved in building and training an RNN.

In [2]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt



In [5]:
# Sample text
text = "hello world"

# Create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Convert text to integers
text_as_int = np.array([char2idx[c] for c in text])

# Show the first characters and their indices
print('{')
for char,_ in zip(char2idx, range(len(char2idx))):
    print(f'  {repr(char)}: {char2idx[char]}')
print('}\n')
print(f'{text} ----> {text_as_int}')


{
  ' ': 0
  'd': 1
  'e': 2
  'h': 3
  'l': 4
  'o': 5
  'r': 6
  'w': 7
}

hello world ----> [3 2 4 4 5 0 7 5 6 4 1]


In [6]:
# Length of each sequence
seq_length = 4
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
sequences = []
for i in range(examples_per_epoch):
    start_index = i * (seq_length + 1)
    end_index = start_index + seq_length + 1
    seq = text_as_int[start_index:end_index]
    sequences.append(seq)

# Split sequences into input and target
inputs = np.array([seq[:-1] for seq in sequences])
targets = np.array([seq[1:] for seq in sequences])

# Convert to PyTorch tensors
inputs = torch.tensor(inputs, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Print the first example
print(f'Input data: {inputs[0]}, Target data: {targets[0]}')


Input data: tensor([3, 2, 4, 4]), Target data: tensor([2, 4, 4, 5])


In [7]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

# Hyperparameters
vocab_size = len(vocab)
embedding_dim = 8
hidden_dim = 16
output_dim = vocab_size

# Instantiate the model
model = RNN(vocab_size, embedding_dim, hidden_dim, output_dim)
print(model)


RNN(
  (embedding): Embedding(8, 8)
  (rnn): RNN(8, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=8, bias=True)
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
# Number of epochs
EPOCHS = 100

for epoch in range(EPOCHS):
    for i in range(len(inputs)):
        input_seq = inputs[i].unsqueeze(0)  # Add batch dimension
        target_seq = targets[i]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(input_seq)
        loss = criterion(output.squeeze(), target_seq)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item()}')

print('Finished Training')


Epoch 1/100, Loss: 2.225950241088867
Epoch 11/100, Loss: 1.9889894723892212
Epoch 21/100, Loss: 1.7738288640975952
Epoch 31/100, Loss: 1.5753997564315796
Epoch 41/100, Loss: 1.3907095193862915
Epoch 51/100, Loss: 1.210460901260376
Epoch 61/100, Loss: 1.03239905834198
Epoch 71/100, Loss: 0.8655322790145874
Epoch 81/100, Loss: 0.7178366184234619
Epoch 91/100, Loss: 0.5922159552574158
Finished Training


In [16]:
def generate_text(model, start_string, num_generate=10):
    input_eval = torch.tensor([char2idx[s] for s in start_string], dtype=torch.long).unsqueeze(0)

    text_generated = []
    model.eval()
    with torch.no_grad():
        for _ in range(num_generate):
            output = model(input_eval)
            predicted_id = torch.argmax(output[0, -1, :]).item()
            input_eval = torch.cat((input_eval, torch.tensor([[predicted_id]], dtype=torch.long)), dim=1)
            text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text
print(generate_text(model, start_string="helo", num_generate=10))


heloollorlorlo


## Conclusion:
In this simple project, following has been done:

Prepared a small text dataset.
Built an RNN model using PyTorch.
Trained the model on the text data.
Generated new text based on the learned patterns.